<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>False Political Claim Detection</h1>
    <h3>Carga y limpieza de los datos - Transformers</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="./images/logo.jpg" alt="UPM" />
</header>

# Índice

1. [Importar librerias](#1.-Importar-Librerias)  
2. [Funciones auxiliares](#2.-Funciones-auxiliares)  
3. [Carga del Dataset](#3.-Carga-del-Dataset)  
4. [Configuracion Statements](#4.-Configuracion-Statements)
   * 4.1 [Configuracion Dataset inicial](#4.1-Configuracion-Dataset-inicial)
   * 4.2 [Speaker y statement](#4.2-Speaker-y-statement)
   * 4.3 [Speaker, statement y subject](#4.3-Speaker,-statement-y-subject)
   * 4.4 [Speaker, party affiliation y statement](#4.4-Speaker,-party-affiliation-y-statement)
   * 4.5 [Speaker, party affiliation, statement y subject](#4.5-Speaker,-party-affiliation,-statement-y-subject)
   * 4.6 [Speaker job, speaker, party affiliation y statement](#4.6-Speaker-job,-speaker,-party-affiliation-y-statement)
   * 4.7 [Speaker job, speaker, party affiliation, statement y subject](#4.7-Speaker-job,-speaker,-party-affiliation,-statement-y-subject)  
6. [Exportar CSV](#5.-Exportar-CSV)  
7. [Referencias](#6.-Referencias)

# 1. Importar Librerias

In [1]:
import pandas as pd

# 2. Funciones auxiliares

Función para crear los diferentes *prompt*

In [50]:
def crear_frase_detallada(selected_columns, row):
    partes = []
    
    if 'speaker' in selected_columns and pd.notna(row['speaker']):
        partes.append(f"{row['speaker']}")
    
    if 'speaker_job' in selected_columns and pd.notna(row['speaker_job']):
        partes.append(f"({row['speaker_job']})")
    
    if 'state_info' in selected_columns and pd.notna(row['state_info']):
        partes.append(f"from {row['state_info']}")
    
    if 'party_affiliation' in selected_columns and pd.notna(row['party_affiliation']):
        partes.append(f"(affiliated with the {row['party_affiliation']} party)")
    
    if 'statement' in selected_columns and pd.notna(row['statement']):
        partes.append(f"said: \"{row['statement']}\"")
    
    if 'subject' in selected_columns and pd.notna(row['subject']):
        partes.append(f"The statement concerns the topics of: {row['subject']}.")
    
    return " ".join(partes)

if __name__ == "__main__":
    # Load the data
    examples = [
        {
            "speaker": "John Doe",
            "speaker_job": "Senator",
            "state_info": "California",
            "party_affiliation": "Democratic",
            "statement": "We need to address climate change.",
            "subject": "Climate Change"
        },
        {
            "speaker": "Jane Smith",
            "speaker_job": "Representative",
            "state_info": "Texas",
            "party_affiliation": "Republican",
            "statement": "The economy is recovering.",
            "subject": "Economy"
        }
    ]
    
    # Select relevant columns
    selected_columns = ['speaker', 'speaker_job', 'state_info', 'party_affiliation', 'statement', 'subject']

    # final_sentences = crear_frase_detallada(selected_columns, examples[0])
    for example in examples:
        final_sentences = crear_frase_detallada(selected_columns, example)
        print(final_sentences)

John Doe (Senator) from California (affiliated with the Democratic party) said: "We need to address climate change." The statement concerns the topics of: Climate Change.
Jane Smith (Representative) from Texas (affiliated with the Republican party) said: "The economy is recovering." The statement concerns the topics of: Economy.


# 3. Carga del Dataset

In [51]:
# Train
url = "data/train.csv" 
df_train = pd.read_csv(url)


# Test
url2 = "data/test_nolabel.csv"
df_test = pd.read_csv(url2)

# 4. Configuracion Statements  

Creación de los distintos *prompts* según las variables que deseemos incorporar.

## 4.1 Configuracion Dataset inicial

In [52]:
df_train_hugging = pd.DataFrame()
df_test_hugging = pd.DataFrame()

In [53]:
df_train_hugging = df_train[['id', 'label', 'statement']]
df_test_hugging = df_test[['id', 'statement']]

In [54]:
df_train.columns

Index(['id', 'label', 'statement', 'subject', 'speaker', 'speaker_job',
       'state_info', 'party_affiliation'],
      dtype='object')

## 4.2 Speaker y statement

In [55]:
columns1 = ['speaker', 'statement']

df_train_hugging['speaker_statement'] = df_train.apply(lambda row: crear_frase_detallada(columns1, row), axis=1)
df_test_hugging['speaker_statement'] = df_test.apply(lambda row: crear_frase_detallada(columns1, row), axis=1)

/tmp/ipykernel_1347289/3924683873.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_hugging['speaker_statement'] = df_train.apply(lambda row: crear_frase_detallada(columns1, row), axis=1)
/tmp/ipykernel_1347289/3924683873.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_hugging['speaker_statement'] = df_test.apply(lambda row: crear_frase_detallada(columns1, row), axis=1)


In [56]:
print(df_train_hugging['speaker_statement'][1])

chris-dodd said: "With the resources it takes to execute just over three months of the Iraq war, we could fully fund the expansion of health care for needy children that Bush vetoed."


## 4.3 Speaker, statement y subject

In [57]:
columns2 = ['speaker', 'statement', 'subject']

df_train_hugging['speaker_statement_subject'] = df_train.apply(lambda row: crear_frase_detallada(columns2, row), axis=1)
df_test_hugging['speaker_statement_subject'] = df_test.apply(lambda row: crear_frase_detallada(columns2, row), axis=1)


/tmp/ipykernel_1347289/4077517249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_hugging['speaker_statement_subject'] = df_train.apply(lambda row: crear_frase_detallada(columns2, row), axis=1)
/tmp/ipykernel_1347289/4077517249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_hugging['speaker_statement_subject'] = df_test.apply(lambda row: crear_frase_detallada(columns2, row), axis=1)


In [58]:
print(df_train_hugging['speaker_statement_subject'][1])

chris-dodd said: "With the resources it takes to execute just over three months of the Iraq war, we could fully fund the expansion of health care for needy children that Bush vetoed." The statement concerns the topics of: health-care.


## 4.4 Speaker, party affiliation y statement

In [59]:
columns3 = ['speaker', 'party_affiliation', 'statement']

df_train_hugging['speaker_party_statement'] = df_train.apply(lambda row: crear_frase_detallada(columns3, row), axis=1)
df_test_hugging['speaker_party_statement'] = df_test.apply(lambda row: crear_frase_detallada(columns3, row), axis=1)

/tmp/ipykernel_1347289/1926735484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_hugging['speaker_party_statement'] = df_train.apply(lambda row: crear_frase_detallada(columns3, row), axis=1)
/tmp/ipykernel_1347289/1926735484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_hugging['speaker_party_statement'] = df_test.apply(lambda row: crear_frase_detallada(columns3, row), axis=1)


In [60]:
print(df_train_hugging['speaker_party_statement'][1])

chris-dodd (affiliated with the democrat party) said: "With the resources it takes to execute just over three months of the Iraq war, we could fully fund the expansion of health care for needy children that Bush vetoed."


## 4.5 Speaker, party affiliation, statement y subject

In [61]:
columns4 = ['speaker', 'party_affiliation', 'statement', 'subject']

df_train_hugging['speaker_party_statement_subject'] = df_train.apply(lambda row: crear_frase_detallada(columns4, row), axis=1)
df_test_hugging['speaker_party_statement_subject'] = df_test.apply(lambda row: crear_frase_detallada(columns4, row), axis=1)

/tmp/ipykernel_1347289/2624612438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_hugging['speaker_party_statement_subject'] = df_train.apply(lambda row: crear_frase_detallada(columns4, row), axis=1)
/tmp/ipykernel_1347289/2624612438.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_hugging['speaker_party_statement_subject'] = df_test.apply(lambda row: crear_frase_detallada(columns4, row), axis=1)


In [62]:
print(df_train_hugging['speaker_party_statement_subject'][1])

chris-dodd (affiliated with the democrat party) said: "With the resources it takes to execute just over three months of the Iraq war, we could fully fund the expansion of health care for needy children that Bush vetoed." The statement concerns the topics of: health-care.


## 4.6 Speaker job, speaker, party affiliation y statement

In [63]:
columns5 = ['speaker', 'speaker_job', 'party_affiliation', 'statement']

df_train_hugging['speaker_speaker-job_party_statement'] = df_train.apply(lambda row: crear_frase_detallada(columns5, row), axis=1)
df_test_hugging['speaker_speaker-job_party_statement'] = df_test.apply(lambda row: crear_frase_detallada(columns5, row), axis=1)

In [64]:
print(df_train_hugging['speaker_speaker-job_party_statement'][1])

chris-dodd (U.S. senator) (affiliated with the democrat party) said: "With the resources it takes to execute just over three months of the Iraq war, we could fully fund the expansion of health care for needy children that Bush vetoed."


## 4.7 Speaker job, speaker, party affiliation, statement y subject

In [65]:
columns6 = ['speaker', 'speaker_job', 'party_affiliation', 'statement', 'subject']

df_train_hugging['speaker_speaker-job_party_statement_subject'] = df_train.apply(lambda row: crear_frase_detallada(columns6, row), axis=1)
df_test_hugging['speaker_speaker-job_party_statement_subject'] = df_test.apply(lambda row: crear_frase_detallada(columns6, row), axis=1)

In [66]:
print(df_train_hugging['speaker_speaker-job_party_statement_subject'][1])

chris-dodd (U.S. senator) (affiliated with the democrat party) said: "With the resources it takes to execute just over three months of the Iraq war, we could fully fund the expansion of health care for needy children that Bush vetoed." The statement concerns the topics of: health-care.


# 5. Exportar CSV

In [67]:
df_train_hugging.to_csv('./formated/train_hugging.csv', index=False)

df_test_hugging.to_csv('./formated/test_hugging.csv', index=False)

# 6. Referencias

* [pandas documentation — pandas 2.2.3 documentation. (s. f.).](https://pandas.pydata.org/docs/)  
